# 최종 코드

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
import pandas as pd
from pykrx import stock
from datetime import datetime



# KOSPI와 KOSDAQ 모든 업종을 가져옵니다.
all_industries = stock.get_index_ticker_list(market="KRX")
all_industries = ['5043', '5044', '5045', '5046', '5048', '5049', '5051', '5052', '5054', '5055', '5056', '5057', '5061', '5062', '5063', '5064', '5065', '5302']


def get_top_n_codes(sosok=0, n=20):

    url = f"https://finance.naver.com/sise/sise_market_sum.naver?sosok={sosok}"
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    response.encoding = 'euc-kr'
    
    # BeautifulSoup로 HTML 파싱
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 시총 테이블 추출
    table = soup.find('table', class_='type_2')
    
    # 테이블의 각 행에서 종목명과 종목 코드 추출
    rows = table.find_all('tr')[2:]  # 상단 2줄은 헤더이므로 제외
    data = []
    
    for row in rows:
        cols = row.find_all('td')
        if len(cols) > 1:
            name = cols[1].text.strip()  
            code = cols[1].find('a')['href'].split('=')[-1]  # 예: "/item/main.naver?code=005930" → "005930"
            data.append((name, code))
    
    # 상위 n개만 DataFrame
    df_top_n = pd.DataFrame(data, columns=['Name', 'Code']).head(n)
    return df_top_n

# 1) KOSPI 상위 20개
df_kospi_top20 = get_top_n_codes(sosok=0, n=20)
# 2) KOSDAQ 상위 20개
df_kosdaq_top20 = get_top_n_codes(sosok=1, n=20)

# 3) 합치기
df_top_40 = pd.concat([df_kospi_top20, df_kosdaq_top20], ignore_index=True)

# 종목 코드 리스트
codes_list = df_top_40['Code'].tolist()
print(codes_list)

# ✅ Step 3: 업종별로 기업 코드 매칭
industry_mapping = {}

for industry_code in all_industries:
    constituents = stock.get_index_portfolio_deposit_file(industry_code)
    industry_name = stock.get_index_ticker_name(industry_code)

    # 상위 10개 기업 코드 중 해당 업종에 속한 기업을 찾기
    matching_codes = [code for code in codes_list if code in constituents]

    if matching_codes:
        industry_mapping[industry_name] = matching_codes

# ✅ 결과 출력
for industry, codes in industry_mapping.items():
    print(f"{industry}: {codes}")


# Start and end dates
start_date = "20241001"
end_date = "20250110"

# Calculate the difference in days and convert to years and months
start_dt = datetime.strptime(start_date, "%Y%m%d")
end_dt = datetime.strptime(end_date, "%Y%m%d")
date_difference = end_dt - start_dt

# Convert the difference into a human-readable format (years and months)
years = date_difference.days // 365
months = (date_difference.days % 365) // 30

# Format the time period string
if years > 0 and months > 0:
    period_str = f"{years}년 {months}개월간"
elif years > 0:
    period_str = f"{years}년간"
else:
    period_str = f"{months}개월간"


# ✅ Step 3: 업종별 외국인 거래대금 계산
industry_foreign_value = {}

for industry, codes in industry_mapping.items():
    total_foreign_value = 0
    
    for code in codes:
        # 외국인의 거래대금 데이터 가져오기
        value_data = stock.get_market_trading_value_by_investor(start_date, end_date, code)
        foreign_value = value_data.loc["외국인"]["순매수"]
        total_foreign_value += foreign_value
    
    # 업종별 총 거래대금 저장
    industry_foreign_value[industry] = total_foreign_value

# ✅ Step 4: 업종별 총 거래대금을 기준으로 내림차순 정렬
sorted_industries = sorted(industry_foreign_value.items(), key=lambda x: x[1], reverse=True)


# ✅ 결과 출력
print_statement = f"\n{period_str} 업종별 외국인 총 순매수 내역:"
print(print_statement)
for industry, value in sorted_industries:
    print(f"{industry}: {value:,.0f} 원")

['005930', '000660', '373220', '207940', '005380', '000270', '068270', '005935', '105560', '035420', '329180', '055550', '012330', '005490', '028260', '138040', '096770', '032830', '012450', '011200', '196170', '247540', '028300', '086520', '277810', '141080', '000250', '214150', '145020', '348370', '058470', '035900', '214450', '065350', '950160', '403870', '068760', '087010', '039030', '257720']
KRX 자동차: ['005380', '000270', '012330']
KRX 반도체: ['005930', '000660', '058470', '403870', '039030']
KRX 헬스케어: ['207940', '068270', '196170', '028300', '141080', '000250', '214150', '145020', '214450', '068760', '087010']
KRX 은행: ['105560', '055550']
KRX 에너지화학: ['096770', '348370']
KRX 철강: ['005490']
KRX 기계장비: ['373220', '329180', '247540', '086520', '277810', '065350']
KRX 보험: ['032830']
KRX 운송: ['011200']
KRX 경기소비재: ['257720']
KRX 미디어&엔터테인먼트: ['035420', '035900']
KRX 정보기술: ['005930', '000660', '058470', '403870', '039030']
코리아 밸류업 지수: ['005930', '000660', '005380', '000270', '068270', '10556

KeyError: '거래대금'

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
from pykrx import stock
from datetime import datetime

# (A) 억/조 단위로 변환해주는 헬퍼 함수
def format_korean_won(value):
    """
    숫자를 받아서, 1억 이상이면 'xx.xx억 원',
    1조 이상이면 'xx.xx조 원',
    그 미만이면 'xx,xxx 원' 형태로 반환합니다.
    """
    sign = "-" if value < 0 else ""
    abs_value = abs(value)
    
    if abs_value >= 1e12:  # 1조 = 1e12
        return f"{sign}{abs_value / 1e12:.2f}조 원"
    elif abs_value >= 1e8:  # 1억 = 1e8
        return f"{sign}{abs_value / 1e8:.2f}억 원"
    else:
        return f"{sign}{abs_value:,.0f} 원"


# (B) KOSPI·KOSDAQ 상위 기업 코드 가져오기 ----------------------
def get_top_n_codes(sosok=0, n=20):
    """
    네이버 금융에서 시가총액 상위 n개 종목을 (이름, 코드) 형태로 반환
    sosok=0 이면 코스피, 1 이면 코스닥
    """
    url = f"https://finance.naver.com/sise/sise_market_sum.naver?sosok={sosok}"

    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
            " AppleWebKit/537.36 (KHTML, like Gecko)"
            " Chrome/108.0.0.0 Safari/537.36"
        )
    }
    response = requests.get(url, headers=headers)
    response.encoding = 'euc-kr'  # 네이버 금융은 euc-kr 인코딩

    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', class_='type_2')

    rows = table.find_all('tr')[2:]  # 상단 2줄은 헤더

    data = []
    for row in rows:
        cols = row.find_all('td')
        if len(cols) > 1:
            name = cols[1].text.strip()
            code = cols[1].find('a')['href'].split('=')[-1]
            data.append((name, code))

    df_top_n = pd.DataFrame(data, columns=['Name', 'Code']).head(n)
    return df_top_n


# ------------------ 실제 코드 시작 ------------------

# 1) KOSPI 상위 20개
df_kospi_top20 = get_top_n_codes(sosok=0, n=20)
# 2) KOSDAQ 상위 20개
df_kosdaq_top20 = get_top_n_codes(sosok=1, n=20)

# 3) 합치기
df_top_40 = pd.concat([df_kospi_top20, df_kosdaq_top20], ignore_index=True)
codes_list = df_top_40['Code'].tolist()

# 가져올 업종 코드 목록
all_industries = [
    '5043', '5044', '5045', '5046', '5048', '5049',
    '5051', '5052', '5054', '5055', '5056', '5057',
    '5061', '5062', '5063', '5064', '5065', '5302'
]

# (C) 업종별 매핑: 코드 -> 기업이름
industry_mapping = {}

for industry_code in all_industries:
    constituents = stock.get_index_portfolio_deposit_file(industry_code)
    industry_name = stock.get_index_ticker_name(industry_code)

    # codes_list 중에서 해당 업종에 포함되는 종목만
    matching_codes = [code for code in codes_list if code in constituents]
    
    # 여기서 '종목 코드' 대신 '종목 이름'을 리스트에 담아 저장
    matching_names = [stock.get_market_ticker_name(code) for code in matching_codes]

    if matching_names:
        industry_mapping[industry_name] = matching_names

# 업종별 매핑 출력 예시
for industry, names in industry_mapping.items():
    print(f"{industry}: {names}")


# (D) 날짜 설정
start_date = "20241001"
end_date = "20250107"

start_dt = datetime.strptime(start_date, "%Y%m%d")
end_dt = datetime.strptime(end_date, "%Y%m%d")
date_difference = end_dt - start_dt

years = date_difference.days // 365
months = (date_difference.days % 365) // 30

if years > 0 and months > 0:
    period_str = f"{years}년 {months}개월간"
elif years > 0:
    period_str = f"{years}년간"
else:
    period_str = f"{months}개월간"


# (E) 업종별 외국인 순매수액 계산
industry_foreign_value = {}

for industry_code in all_industries:
    # 실제 계산은 여전히 '코드'가 필요하므로,
    # industry_mapping (이름 리스트) 대신 다시 코드 리스트를 구해야 합니다.
    # (이미 매칭된 '기업 이름'만 있는 경우엔 이름 -> 코드 변환 작업이 필요합니다.)
    # 하지만 여기서는 간단히, 전체 constituents 중에서 상위40 코드(codes_list)에 해당하는 것만 계산한다 가정.
    
    # 우선 해당 업종 종목 목록(코드)을 얻어옵니다.
    constituents = stock.get_index_portfolio_deposit_file(industry_code)
    # 상위 40개 중 이 업종에 해당하는 종목들
    matching_codes = [code for code in codes_list if code in constituents]
    
    if not matching_codes:
        continue  # 해당 업종에 해당하는 종목이 없으면 건너뜀
    
    industry_name = stock.get_index_ticker_name(industry_code)

    # 업종별 순매수 합
    total_foreign_value = 0
    for code in matching_codes:
        # 외국인의 순매수 거래대금
        value_data = stock.get_market_trading_value_by_investor(start_date, end_date, code)
        # 혹시 데이터가 없거나 에러가 날 경우를 대비
        if "외국인" in value_data.index:
            foreign_net_value = value_data.loc["외국인"]["순매수"]
            total_foreign_value += foreign_net_value
    
    industry_foreign_value[industry_name] = total_foreign_value

# (F) 업종별 순매수액 내림차순 정렬
sorted_industries = sorted(industry_foreign_value.items(), key=lambda x: x[1], reverse=True)

# (G) 결과 출력 (억/조 단위로 포맷)
print_statement = f"\n{period_str} 업종별 외국인 총 순매수 내역:"
print(print_statement)
for industry, value in sorted_industries:
    print(f"{industry}: {format_korean_won(value)}")


['005380', '000270', '012330', '161390', '018880', '204320', '000240', '005850', '073240', '004490', '007340', '011210', '003620', '064960', '271940', '009900', '101360', '015750', '200880', '010690']


KeyError: '지수명'

In [16]:
import matplotlib.pyplot as plt

# 예: 이미 구해놓은 industry_foreign_value 딕셔너리
# industry_foreign_value = {
#     'KRX 반도체': 6636799000000,
#     'KRX 정보기술': 6636799000000,
#     ...
# }

# 순매수 금액 기준 내림차순 정렬
sorted_items = sorted(industry_foreign_value.items(), key=lambda x: x[1], reverse=True)
industries = [item[0] for item in sorted_items]
values = [item[1] for item in sorted_items]

# 양수=초록색, 음수=빨간색
colors = ['green' if v >= 0 else 'red' for v in values]

plt.figure(figsize=(19, 8))

bars = plt.barh(industries, values, color=colors)
plt.gca().invert_yaxis()        # y축 뒤집기 (위→아래 순서로)
plt.axvline(x=0, color='black') # x=0 선

# ─────────────────────────────────────────────────────────
# (A) x축 범위를 넉넉히 잡아 텍스트가 안 잘리도록 설정
# ─────────────────────────────────────────────────────────
min_val = min(values)
max_val = max(values)
range_val = max_val - min_val

# 그래프 좌우에 5% 정도의 여유를 둠
margin = 0.05 * range_val
left_lim = min_val - margin
right_lim = max_val + margin
plt.xlim(left_lim, right_lim)

# ─────────────────────────────────────────────────────────
# (B) 막대 끝에 표시할 텍스트를 막대에서 조금 떼어줌(오프셋)
# ─────────────────────────────────────────────────────────
for bar, val in zip(bars, values):
    width = bar.get_width()
    y_pos = bar.get_y() + bar.get_height()/2
    
    # 표시할 텍스트
    label_text = format_korean_won(val)
    
    # (1) 양수일 때는 막대 오른쪽에, 음수일 때는 막대 왼쪽에  
    # (2) margin의 일부를 '오프셋'으로 사용
    offset = 0.01 * abs(range_val)  # x축 전체 범위 중 1% 정도를 오프셋으로
    
    if val >= 0:
        plt.text(width + offset,
                 y_pos,
                 label_text,
                 va='center',
                 ha='left',
                 fontsize=10)
    else:
        plt.text(width - offset,
                 y_pos,
                 label_text,
                 va='center',
                 ha='right',
                 fontsize=10)

min_val = min(values)
max_val = max(values)
range_val = max_val - min_val

# 예: x축 양옆에 10% 정도 여유를 둠
margin_ratio = 0.10  
margin = margin_ratio * range_val

# x축 범위 설정
plt.xlim(min_val - margin, max_val + margin)

# 제목, 라벨
plt.title(f"{period_str} 업종별 외국인 순매수 내역 (단위 변환: 억/조)")
plt.xlabel("순매수 금액 (원)")

# tight_layout으로 Figure 여백 조절
plt.tight_layout()
plt.show()


NameError: name 'industry_foreign_value' is not defined